# Imports and Setups

In [1]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.5 MB/s eta 0:00:00


In [2]:
! pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 4.5 MB/s eta 0:00:00


In [3]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.3 MB/s eta 0:00:00


Lendo o Arquivo

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [6]:
num_gen = 5
perc = 5

In [7]:
from nlpaug.augmenter.word import WordAugmenter

class M2M100BackTranslationAug(WordAugmenter):


    def __init__(self, from_lang='en', to_lang='pt',n_data = 1, num_beams = 10,
        name='M2M100BackTranslationAug'):
        super().__init__(
            action='substitute')

        from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

        model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to('cuda')
        tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

        self.src = from_lang
        self.dest = to_lang
        self.model = model
        self.tokenizer = tokenizer
        self.n_data = n_data
        self.num_beams = num_beams

    def substitute(self, data, n=1):

        self.tokenizer.src_lang = self.src
        encoded_hi = self.tokenizer(data, return_tensors="pt", truncation=True, padding=True).to('cuda')
        generated_tokens = self.model.generate(**encoded_hi, forced_bos_token_id=self.tokenizer.get_lang_id(self.dest),
                                               num_return_sequences = self.n_data, num_beams = self.num_beams)
        resultado = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

        self.tokenizer.src_lang = self.dest
        encoded_hi2 = self.tokenizer(resultado, return_tensors="pt", truncation=True, padding=True).to('cuda')
        generated_tokens = self.model.generate(**encoded_hi2, forced_bos_token_id=self.tokenizer.get_lang_id(self.src))
        texto = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

        return texto

In [8]:
import nlpaug.augmenter.word as naw

augmenter = M2M100BackTranslationAug(from_lang='en', to_lang='cs', n_data=5, num_beams=10)

In [9]:
augmenter.augment("i love to travel to D.C.")

[['I like traveling to D.C.',
  'I like to travel to D.C.',
  'I love traveling to D.C.',
  'I love traveling to D.C.',
  'I love traveling to D.C.']]

# Emotion

In [ ]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/base_emotions_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))
  export = base_cru_1p.copy()
  export = export.explode('generated').reset_index(drop=True)
  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/back_translation/base_emotions_'+str(perc)+'_protaugment8_'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

100%|██████████| 800/800 [19:19<00:00,  1.45s/it]


# Emoint

In [ ]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/base_emoint_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))
  export = base_cru_1p.copy()
  export = export.explode('generated').reset_index(drop=True)
  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/back_translation/base_emoint_'+str(perc)+'_protaugment8_'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

100%|██████████| 163/163 [03:58<00:00,  1.46s/it]


# Offensive

In [11]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))
  export = base_cru_1p.copy()
  export = export.explode('generated').reset_index(drop=True)
  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/back_translation/base_offensive_'+str(perc)+'_protaugment8_'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

100%|██████████| 170/170 [05:49<00:00,  2.06s/it]


# Ironic

In [12]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))
  export = base_cru_1p.copy()
  export = export.explode('generated').reset_index(drop=True)
  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/back_translation/base_ironic_'+str(perc)+'_protaugment8_'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

100%|██████████| 144/144 [03:28<00:00,  1.45s/it]


# Tweet_Sentiment

In [ ]:
nums = [1,2,3,4,5]
for i in nums:
  arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i)+'.csv'
  base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
  base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))
  export = base_cru_1p.copy()
  export = export.explode('generated').reset_index(drop=True)
  arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/back_translation/base_tweet_sentiment_'+str(perc)+'_protaugment8_'+str(i)+'.csv'
  export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

100%|██████████| 210/210 [06:20<00:00,  1.62s/it]

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))

100%|██████████| 4000/4000 [4:04:15<00:00,  3.66s/it]


In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

,texto,label,generated
0,i also feel a little selfish when i get excite...,3,[I also feel a little selfish when I feel exci...
1,i also feel a little selfish when i get excite...,3,[I also feel a little selfish when I feel exci...
2,i also feel a little selfish when i get excite...,3,[I also feel a little selfish when I feel exci...
3,i also feel a little selfish when i get excite...,3,[I also feel a little selfish when I feel exci...
4,i feel it is equally important that you know i...,1,[I feel it is equally important that you know ...


In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# TREC

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_trec_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')
base_cru_1p['indices'] = [i for i in range(base_cru_1p.shape[0])]

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))

100%|█████████▉| 1361/1363 [48:31<00:04,  2.15s/it]

In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_trec_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# Offensive

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_offensive_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))

100%|██████████| 2979/2979 [3:54:19<00:00,  4.72s/it]



In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

,texto,label,generated
0,@user Childess,0,"[User Childhood, User Childhood, Users of Chil..."
1,@user Childess,0,"[User Childhood, User Childhood, Users of Chil..."
2,@user Childess,0,"[User Childhood, User Childhood, Users of Chil..."
3,@user Childess,0,"[User Childhood, User Childhood, Users of Chil..."
4,Confirm the MAN! THE LEFT WING ACTIVIST IS A L...,1,[Confirm men! the left activist is a liar whos...


,texto,label,generated
0,@user Childess,0,"[User Childhood, User Childhood, Users of Chil..."
1,@user Childess,0,"[User Childhood, User Childhood, Users of Chil..."
2,@user Childess,0,"[User Childhood, User Childhood, Users of Chil..."
3,@user Childess,0,"[User Childhood, User Childhood, Users of Chil..."
4,Confirm the MAN! THE LEFT WING ACTIVIST IS A L...,1,[Confirm men! the left activist is a liar whos...


In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_offensive_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# Ironic

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))

100%|██████████| 716/716 [40:20<00:00,  3.38s/it]



In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

,texto,label,generated
0,@user @user but $7 at a big guy like sephora o...,0,[@user @user but $7 in a big guy like sephora ...
1,@user @user but $7 at a big guy like sephora o...,0,[@user @user but $7 in a big guy like sephora ...
2,@user @user but $7 at a big guy like sephora o...,0,[@user @user but $7 in a big guy like sephora ...
3,@user @user but $7 at a big guy like sephora o...,0,[@user @user but $7 in a big guy like sephora ...
4,"Enroll 2 Members 1 on each leg (L/R). THEN, he...",0,[Subscribe 2 Members 1 on each foot (L/R). THE...


,texto,label,generated
0,@user @user but $7 at a big guy like sephora o...,0,[@user @user but $7 in a big guy like sephora ...
1,@user @user but $7 at a big guy like sephora o...,0,[@user @user but $7 in a big guy like sephora ...
2,@user @user but $7 at a big guy like sephora o...,0,[@user @user but $7 in a big guy like sephora ...
3,@user @user but $7 at a big guy like sephora o...,0,[@user @user but $7 in a big guy like sephora ...
4,"Enroll 2 Members 1 on each leg (L/R). THEN, he...",0,[Subscribe 2 Members 1 on each foot (L/R). THE...


In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# Emoint

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_emoint_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))

100%|██████████| 815/815 [13:09<00:00,  1.03it/s]


In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

,texto,label,generated
0,Sounds like Donald Trump has spent today just ...,0,Sounds like Donald Trump spent today just doin...
1,Sounds like Donald Trump has spent today just ...,0,Sounds like Donald Trump spent today just doin...
2,Sounds like Donald Trump has spent today just ...,0,Sounds like Donald Trump spent today just doin...
3,Sounds like Donald Trump has spent today just ...,0,Sounds like Donald Trump spent today just doin...
4,also i had an awful nightmare involving being ...,3,I also had a terrible nightmare that included ...


In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_emoint_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# Emoji

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_emoji_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))

100%|██████████| 11250/11250 [2:32:11<00:00,  1.23it/s]


In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

,texto,label,generated
0,Late meet up.... Fresh from Sweden #friendship...,9,Later we’ll meet... Light from Sweden #friends...
1,Late meet up.... Fresh from Sweden #friendship...,9,We'll see you later.... Light from Sweden #fri...
2,Late meet up.... Fresh from Sweden #friendship...,9,Lights from Sweden #friendshipforever
3,Late meet up.... Fresh from Sweden #friendship...,9,Lights from Sweden #friendshipforever
4,A big thank you to all the @user sponsors! Tha...,0,Thank you very much to all @user sponsors! tha...


In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_emoji_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# Rotten

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_rotten_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))

100%|██████████| 2133/2133 [38:01<00:00,  1.07s/it]


In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

,texto,label,generated
0,the spark of special anime magic here is unmis...,1,Spark special anime magic here is meaningless ...
1,the spark of special anime magic here is unmis...,1,Spark special anime magic here is meaningless ...
2,the spark of special anime magic here is unmis...,1,Spark special anime magic here is unmistakable...
3,the spark of special anime magic here is unmis...,1,Spark special anime magic here is unmistakable...
4,it's as if you're watching a movie that was ma...,0,It’s like watching a film that was made in 197...


In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_rotten_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# IMDB

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_imdb_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x, n=num_gen))

In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_imdb_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# AGNEWS

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_agnews_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x, n=num_gen))

In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,generated,texto,label
0,and this a given new standard trade in great b...,Great White Sharks Given New Protection Trade ...,0
1,great white sharks given new protection trade ...,Great White Sharks Given New Protection Trade ...,0
2,great white sharks given new protection result...,Great White Sharks Given New Protection Trade ...,0
3,even white sharks given new protection trade f...,Great White Sharks Given New Protection Trade ...,0
4,great white sharks said new protection trade i...,Great White Sharks Given New Protection Trade ...,0


In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_agnews_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

# SST

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_sst_'+str(perc)+'_original1.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [ ]:
base_cru_1p['generated'] = base_cru_1p['texto'].progress_apply(lambda x : augmenter.augment(x))

100%|██████████| 2136/2136 [34:27<00:00,  1.03it/s]


In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)
export.head()

,texto,label,generated
0,Perry 's good and his is an interesting charac...,0,Perry is good and his character is interesting...
1,Perry 's good and his is an interesting charac...,0,Perry is good and his character is interesting...
2,Perry 's good and his is an interesting charac...,0,Perry is good and his character is interesting...
3,Perry 's good and his is an interesting charac...,0,Perry is good and his character is interesting...
4,( A ) poorly executed comedy .,0,A poorly performed comedy.


In [ ]:
i = 0
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/base_sst_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
export.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')